# 1) Download sounds from a id list

## Also download all features, BUT remember it is not frame-based, its the statistical aggregattion

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [1]:
import os
import pandas as pd
import numpy as np
#import freesound # pip install --user git+https://github.com/MTG/freesound-python
import freesound_lluis as freesound # this is my own version, until I can use the PR I made
from IPython.display import display

FREESOUND_API_KEY = 's9xV21k899aPetFLLLj7IQjq2hCprXG0IqymUEy1'  # my api key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [2]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound(freesound_id):
    "retrives sound by id --> https://github.com/MTG/freesound-python/blob/master/freesound.py"
    return freesound_client.get_sound(freesound_id)
    
    
def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def retrieve_analysis_by_frame(sound, features_to_use):
    descriptors = ",".join(features_to_use)
    return sound.get_analysis(descriptors=descriptors)

def make_record(sound, file_path = "files/"): 
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    record['path'] = file_path + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record


In [3]:
#ROUTES

DOWNLOADED_PREVIEWS_DIR = FILES_DIR + '/previews/'
DATAFRAME_CSV_FILENAME = 'dataframes/multiple_events.csv'

In [4]:
#MY OWN CONTRIBUTION: download by ids

# Build our collection of sounds
DOWNLOADED_PREVIEWS_DIR = FILES_DIR + '/previews/'
DATAFRAME_CSV_FILENAME = 'dataframes/multiple_events.csv'

problems=[]

def build_sound_collection(freesound_ids, features_to_use, sound_files_path, dataframe_filename):
    
    anal_by_id = {}
    sounds = []
    records = []
    
    for count, freesound_id in enumerate(freesound_ids):
        try:
            sound = retrieve_sound(freesound_id)
            sounds.append(sound)
            print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(freesound_ids)))
            retrieve_sound_preview(sound, sound_files_path) # Download the sounds and save them to FILES_DIR folder
            '''test_anal = retrieve_analysis_by_frame(sound, features_to_use)
            anal_by_id[sound.id] = test_anal'''

            records = [make_record(s, sound_files_path) for s in sounds]
        except: 
            print("====== problem with", freesound_id)
            problems.append(freesound_id)
    
    # ==> we want to save an .npy file for each freesound_id
    
    # Do all queries and concatenate the results in a single list of sounds
    #sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

    return records, sounds


''' === available lowlevel features for frame analysis: ====
['barkbands_spread', 'spectral_kurtosis', 'scvalleys', 'erb_bands', 'frequency_bands', 'spectral_flatness_db', 
'pitch', 'average_loudness', 'spectral_centroid', 'dissonance', 'spectral_crest', 'spectral_skewness', 
'pitch_instantaneous_confidence', 'barkbands', 'silence_rate_20dB', 'spectral_strongpeak', 'startFrame', 'stopFrame',
'silence_rate_30dB', 'pitch_salience', 'spectral_energyband_high', 'silence_rate_60dB',
'spectral_energyband_middle_high', 'zerocrossingrate', 'barkbands_skewness', 'spectral_entropy', 'hfc', 
'spectral_rolloff', 'spectral_contrast', 'spectral_flux', 'spectral_energy', 'spectral_rms', 'spectral_decrease', 
'spectral_spread', 'spectral_energyband_middle_low', 'barkbands_kurtosis', 'spectral_energyband_low', 
'spectral_complexity', 'mfcc', 'gfcc']
'''


nature_ids = [415297,403357,41135,58155,403358,428266,403359,403357,405373,397092,148870,51667,51643,136971,136692,148879,136691,148877,161013,231275,263810,53469,17736,424692,265034,7744,79232,105383,194993,223277,174624,206154,98779,275468,234107,97389,171473,236952,95892,207473,147182,400384,432846,434352,394959,402547,179900,267464,250725,86567,402801,317130,33256,169693,251635,367328,428266]
human_ids = [424844,412776,278295,192529,186253,46328,46325,94149,58334,259168,350324,132900,251473,60118,58168,44478,136052,341874,57744,55060,122814,401539,401993,130597,424844,72009,115088,205463,249776,75277,241698,69823,138469,240725,80980,101792,56730,210093,192178,17093,191337,365220,44311,371339,397336,161018,153425,239299,233261,169944,340747,329568,387459,407207,387460,397667,45352,31169,25270,32868,46228,49482,42811,39843,113148,407292,334980,211945,457258,32380,350655,383879,25450,326244,259630,203305,233018,407735,327497,366843,365965,145267,332183,369871,407499,389592,344238,24089,324427,81782,201942,346457,278295,192529,186253]
freesound_ids = nature_ids + human_ids


#features_to_use = ['lowlevel.mfcc'] 
features_to_use = [] 

#df, sounds = build_sound_collection(freesound_queries_music_loops, DOWNLOADED_PREVIEWS_DIR, DATAFRAME_CSV_FILENAME_MUSIC_LOOPS)
#df.head(5)

records, sounds = build_sound_collection(freesound_ids, features_to_use, DOWNLOADED_PREVIEWS_DIR, DATAFRAME_CSV_FILENAME)
#download problems
print("try again, with the problems")
records_problems, sounds_problems = build_sound_collection(problems, features_to_use, DOWNLOADED_PREVIEWS_DIR, DATAFRAME_CSV_FILENAME)
records = records + records_problems
sounds = sounds + sounds_problems



====== problem with 148870
try again, with the problems


In [14]:
#save sounds so I dont need to download again:
import pickle

def save_as_binary(element, filename = './files/no_name.file'):
    print('saving as pickle binary ...')
    with open(filename, "wb") as f:
        pickle.dump(element, f, pickle.HIGHEST_PROTOCOL)
    print('... file saved as', filename)

def load_as_binary(filename = './files/no_name.file'):
    with open(filename, "rb") as f:
        data = pickle.load(f)
    return data

save_as_binary(sounds, './temp/sounds.file')

sounds = load_as_binary('./temp/sounds.file')




saving as pickle binary ...
... file saved as ./temp/sounds.file


### Download analysis by frames and save each one to a .npy file

In [22]:
problems = []

def get_analysis_frames_for_collection(sounds, folder_to_save='mfcc'):
    for count, sound in enumerate(sounds):
        try:
            print('Downloading analysis with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
            analysis_frames = sound.get_analysis_frames()
            analysis_frames.lowlevel.mfcc # We only want mfccs at the moment 
            filename = os.path.join(folder_to_save, str(sound.id))
            np.save(filename, analysis_frames.lowlevel.mfcc) #save in file
        except:
            print("====== problem with", sound.id)
            problems.append(sound)
    print("====== DONE")
        
get_analysis_frames_for_collection(sounds)


====== problem with 423398


## Create sounds_metadata.json for use with web-visu
### It follows this structure:
{
    "131086": {
        "username": "ecfike",
        "preview": "https://freesound.org/data/previews/131/131086_2337290-lq.ogg",
        "name": "What's Up Yall 1.wav",
        "tags": ["boy", "male", "whats", "up", "whats-up-yall", "words", "word", "voice", "yall", "man"]
    }
}



In [53]:
import json

def save_to_json(element, filename = './files/no_name.json'):
    print('saving to json ...')
    with open(filename, 'w') as fp:
        json.dump(element, fp, sort_keys=True, indent=4) #pretty json
    print('... file saved as', filename)

    
def create_sounds_metadata(sounds):
    sounds_metadata = {}
    for sound in sounds:
        sounds_metadata[str(sound.id)] = {
            "username": sound.username,
            "preview": sound.previews.preview_lq_ogg,
            "name": sound.name,
            "tags": sound.tags 
        }
    return sounds_metadata

sounds_metadata = create_sounds_metadata(sounds)

save_to_json(sounds_metadata, './files/sounds_metadata_own.json')

saving to json ...
... file saved as ./files/sounds_metadata_own.json
